In [1]:
library(rasterVis)
library(colorRamps)
library(BAMMtools)
library(rgdal)
library(raster)
library(classInt)
library(RColorBrewer)
library(scales)

Loading required package: raster
Loading required package: sp
Loading required package: lattice
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: ape

Attaching package: ‘ape’

The following objects are masked from ‘package:raster’:

    rotate, zoom

rgdal: version: 1.2-18, (SVN revision 718)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.1.3, released 2017/20/01
 Path to GDAL shared files: /Library/Frameworks/R.framework/Versions/3.4/Resources/library/rgdal/gdal
 GDAL binary built with GEOS: FALSE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: /Library/Frameworks/R.framework/Versions/3.4/Resources/library/rgdal/proj
 Linking to sp version: 1.2-7 
Loading required package: spData
To access larger datasets in this package, install the spDataLarge
package with: `install.packages('spDataLarge',
repos='https://nowosad.github.io/

In [2]:
path <- '/Users/dongmeichen/Documents/beetle/data/'
out <- '/Users/dongmeichen/Documents/beetle/images/DEA/'

In [3]:
mpb10km.path <- "/Users/dongmeichen/Documents/beetle/shp"
mpb10km <- readOGR(dsn = mpb10km.path, layer = "mpb10km")
crs <- proj4string(mpb10km)
lonlat <- CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")

OGR data source with driver: ESRI Shapefile 
Source: "/Users/dongmeichen/Documents/beetle/shp", layer: "mpb10km"
with 17 features
It has 2 fields


In [4]:
df2spdf <- function(col1, col2, colname1, colname2, df){
  xy <- data.frame(df[,c(col1,col2)])
  coordinates(xy) <- c(colname1, colname2)
  proj4string(xy) <- lonlat
  xy.n <- spTransform(xy, crs)
  spdf <- SpatialPointsDataFrame(coords = xy.n, data = df, proj4string = crs)
  return(spdf)
}

In [5]:
indata <- read.csv(paste0(path, 'mpb10km_data.csv'))

In [6]:
GAPs <- read.csv(paste0(path, 'mpb10km_GAPs.csv'))
indata <- cbind(indata, GAPs)

In [7]:
# vcc values > 6 indicate areas of no vegetation, replace with 0s
indata$vcc[indata$vcc > 6] <- NA
indata$prs[indata$prs > 20] <- NA
indata$pms[indata$pms > 20] <- NA
indata$pls[indata$pls > 20] <- NA

# Mean fire return interval where there are no trees is effectively 
# infinite--set to 22 (> 1000 years)
indata$mfri[indata$mfri > 22] <- NA

In [8]:
df <- subset(indata, !is.na(beetleAcres))
df$severity <- ifelse(df$prs >= 17 & df$mfri >= 16, 'replacement', ifelse(df$pls >= 17 & df$mfri <= 4, 'low', 'mixed'))
df$severity.no <- ifelse(df$severity == 'replacement', 3, ifelse(df$severity == 'low', 1, 2))
spdf <- df2spdf(1, 2, 'lon', 'lat', df)

In [9]:
mpb10km.pts.r <- raster("/Users/dongmeichen/Documents/beetle/ncfiles/mpb10km_grid.nc", varname = "etopo1")

Loading required namespace: ncdf4


In [10]:
head(spdf)

,lon,lat,etopo1,x,y,beetleAcres,host,forest,mStdAge,density,⋯,SprsCPA,SprsFires,PctSprs,SprsAcre,SprsDays,OutDays,GAPs,wilderness,severity,severity.no
7,-124.1712,40.22942,365.17722,-990000,-460000,0.800,1,1,25.50000,58050.83,⋯,NA,NA,NA,NA,NA,NA,0,0,mixed,2
17,-124.0860,40.42007,493.08517,-980000,-440000,0.800,0,1,42.46217,52809.04,⋯,NA,NA,NA,NA,NA,NA,0,0,mixed,2
19,-124.1177,40.59810,204.08624,-980000,-420000,0.800,0,1,35.00000,50636.57,⋯,538.4615,1,0.5,0.1,2,7,0,0,mixed,2
33,-123.9379,40.25447,487.52399,-970000,-460000,0.800,1,1,169.82977,54385.98,⋯,NA,NA,NA,NA,NA,NA,0,0,mixed,2
34,-123.9535,40.34353,350.69608,-970000,-450000,0.800,0,1,249.11995,47005.25,⋯,NA,NA,NA,NA,NA,NA,2,0,mixed,2
44,-124.2107,41.76738,13.40179,-970000,-290000,1.155,1,0,85.00000,12070.25,⋯,NA,NA,NA,NA,NA,NA,0,0,NA,NA


In [11]:
# functions
mapping.LF <- function(shp, var){
    if(var %in% c('pls', 'pms', 'prs')){
        labels <- c("0-5", "6-10", "11-15", "16-20", "21-25", "26-30",
                    "31-35", "36-40", "41-45", "46-50", "51-55", "56-60",
                    "61-65", "66-70", "71-75", "76-80", "81-85", "86-90", "91-95", "96-100")
    if(var == 'pls'){
        title <- "Percent of low-severity fires"
    }else if(var == 'pms'){
        title <- "Percent of mixed-severity fires"
    }else{
        title <- "Percent of replacement-severity fires"
        labels <- c("0-5", "6-10", "11-15", "16-20", "21-25", "26-30",
                    "31-35", "36-40", "41-45", "46-50", "51-55", "56-60",
                    "61-65", "66-70", "71-75", "76-80", "81-85", "86-90", "96-100")
    }
        cols <- matlab.like(20)
    }else if(var == 'mfri'){
        labels <- c("6-10", "11-15", "16-20", "21-25", "26-30",
                    "31-35", "36-40", "41-45", "46-50", "51-60", "61-70",
                    "71-80", "81-90", "91-100", "101-125", "126-150", "151-200",
                    "201-300", "301-500", "501-1000", ">1000")
        cols <- rev(matlab.like(22))
        title <- "Mean fire return interval"
    }else if(var == 'vcc'){
        labels <- c("0-16", "17-33", "34-50", "51-66", "67-83", "84-100")
        cols <- brewer.pal(7,'BuGn')[-1]
        title <- "Vegetation condition class"
    }
    shp <- shp[!is.na(shp@data[,var]),]
    r <- rasterize(shp, mpb10km.pts.r, var, fun=mean, na.rm=TRUE) 
    r <- as.factor(r)
    rat <- levels(r)[[1]]
    rat[["labels"]] <- labels
    levels(r) <- rat
    p <- levelplot(r, col.regions=cols, xlab="", ylab="",par.settings = list(axis.line = list(col = "transparent")), 
                    scales = list(draw = FALSE), margin=F, main=title)
    p <- p + latticeExtra::layer(sp.polygons(mpb10km, lwd=0.5, col=scales::alpha("black", alpha = 0.6)))
    return(p)
}

In [12]:
mapping.sprs <- function(shp, var, title, cols="YlOrRd"){
    shp <- shp[shp@data[,var] != Inf & !is.na(shp@data[,var]),]
    qt99 <- quantile(shp@data[,var], 0.99)
    shp <- shp[shp@data[,var] <= qt99,]
    r <- rasterize(shp, mpb10km.pts.r, var, fun=mean, na.rm=TRUE)
    ncls <- 6
    brks <- getJenksBreaks(getValues(r), ncls)
    #print(brks)
    p <- levelplot(r, col.regions=brewer.pal(ncls,cols)[-1], cuts=ncls-1, at=brks, xlab="", ylab="", 
                   par.settings = list(axis.line = list(col = "transparent")), 
                   scales = list(draw = FALSE), margin=F, main=title)
    p <- p + latticeExtra::layer(sp.polygons(mpb10km, lwd=0.5, col=scales::alpha("black", alpha = 0.6)))
    return(p)
}

In [13]:
mapping.btl <- function(shp, var, title='MPB affected acres', cols="YlOrRd"){
    r <- rasterize(shp, mpb10km.pts.r, var, fun=mean, na.rm=TRUE)
    ncls <- 6
    brks <- getJenksBreaks(getValues(r), ncls+1)
    p <- levelplot(r, col.regions=brewer.pal(ncls,cols), cuts=ncls, at=brks, xlab="", ylab="", 
                   par.settings = list(axis.line = list(col = "transparent")), 
                    scales = list(draw = FALSE), margin=F, main=title)
    p <- p + latticeExtra::layer(sp.polygons(mpb10km, lwd=0.5, col=scales::alpha("black", alpha = 0.6)))
    return(p)
}

mapping.tree <- function(shp, var, title, cols="PuBuGn"){
    r <- rasterize(shp, mpb10km.pts.r, var, fun=mean, na.rm=TRUE)
    ncls <- 6
    brks <- getJenksBreaks(getValues(r), ncls)
    if(var == 'host' | var == 'forest'){
        labels <- c("No", "Yes")
        cols <- c("#d0d1e6", "#016c59")
        r <- as.factor(r)
        rat <- levels(r)[[1]]
        rat[["labels"]] <- labels
        levels(r) <- rat
        p <- levelplot(r, col.regions=cols, xlab="", ylab="",par.settings = list(axis.line = list(col = "transparent")), 
                        scales = list(draw = FALSE), margin=F, main=title)
    }else{
        p <- levelplot(r, col.regions=brewer.pal(ncls,cols)[-1], cuts=ncls-1, at=brks, xlab="", ylab="", 
                       par.settings = list(axis.line = list(col = "transparent")), 
                        scales = list(draw = FALSE), margin=F, main=title)
    }
    p <- p + latticeExtra::layer(sp.polygons(mpb10km, lwd=0.5, col=scales::alpha("black", alpha = 0.6)))
    return(p)
}

In [14]:
mapping.GAPs <- function(shp, var, title){
    r <- rasterize(shp, mpb10km.pts.r, var, fun=mean, na.rm=TRUE)
    r <- as.factor(r)
    rat <- levels(r)[[1]]
    rat[["labels"]] <- c('0', '1', '2', '3', '4')
    levels(r) <- rat
    p <- levelplot(r, col.regions=c('Grey', brewer.pal(4,'Set1')), xlab="", ylab="",
                    par.settings = list(axis.line = list(col = "transparent")), 
                    scales = list(draw = FALSE), margin=F, main=title)
    p <- p + latticeExtra::layer(sp.polygons(mpb10km, lwd=0.5, col=scales::alpha("black", alpha = 0.6)))
    return(p)
}

In [15]:
mapping.severity <- function(){
    labels <- c("L", "M", "R")
    cols <- c("#e41a1c", "#4daf4a", "#377eb8")
    r <- rasterize(spdf, mpb10km.pts.r, "severity.no", fun=mean, na.rm=TRUE)
    r <- as.factor(r)
    rat <- levels(r)[[1]]
    rat[["labels"]] <- labels
    levels(r) <- rat
    title <- "Fire severity"
    p <- levelplot(r, col.regions=cols, xlab="", ylab="",par.settings = list(axis.line = list(col = "transparent")), 
                    scales = list(draw = FALSE), margin=F, main=title)
    p <- p + latticeExtra::layer(sp.polygons(mpb10km, lwd=0.5, col=scales::alpha("black", alpha = 0.6)))
    return(p)
}

In [17]:
vars <- c("mStdAge", "density", "PctLarge", "PctOld", "forest", 'GAPs')
titles <- c('Stand age', 'Tree density', 'Ratio of large trees', 
            'Ratio of old trees', 'Forested area', 'GAP status')
pos <- cbind(c(1,1),c(1,2),c(1,3),c(2,1),c(2,2),c(2,3))
png(paste0(out,'stand_variable_maps.png'), width=12, height=8, units="in", res=300)
par(mfrow=c(2,3), xpd=FALSE, mar=rep(0.5,4))
for(var in vars){
    if(var != 'GAPs'){
        p <- mapping.tree(spdf, var, titles[which(vars==var)])
    }else{
        p <- mapping.GAPs(spdf, var, 'GAP status')
    }
    print(p,split=c(pos[,which(vars==var)][2], pos[,which(vars==var)][1], 3, 2), newpage=FALSE) 
    print(var)
}
dev.off()

[1] "mStdAge"
[1] "density"
[1] "PctLarge"
[1] "PctOld"
[1] "forest"
[1] "GAPs"


pdf 
  2

In [22]:
# fire regime maps
vars <- c("vcc", "mfri", "prs", "pms", "pls", "severity.no")
pos <- cbind(c(1,1),c(1,2),c(1,3),c(2,1),c(2,2),c(2,3))
png(paste0(out,'fire_regime_variable_maps.png'), width=12, height=8, units="in", res=300)
par(mfrow=c(2,3), xpd=FALSE, mar=rep(0.5,4))
for(var in vars){
    if(var != 'severity.no'){
        p <- mapping.LF(spdf, var)
    }else{
        p <- mapping.severity()
    }
    print(p,split=c(pos[,which(vars==var)][2], pos[,which(vars==var)][1], 3, 2), newpage=FALSE) 
    print(var)
}
dev.off()

[1] "vcc"
[1] "mfri"
[1] "prs"
[1] "pms"
[1] "pls"
[1] "severity.no"


pdf 
  2

In [16]:
# fire data maps
vars <- c('SprsCosts', 'SprsAcres', 'SprsCPA', 'SprsFires', 'PctSprs', 'SprsAcre', 'SprsDays', 'OutDays')
titles <- c('Suppression costs', 'Suppression acres', 'Unit suppression costs', 
            'No. fires suppressed', 'Ratio of suppressed fires',
            'Fire size of suppressed fires', 'Containment duration', 'Fire out duration')

pos <- cbind(c(1,1),c(1,2),c(1,3),c(1,4),c(2,1),c(2,2),c(2,3),c(2,4))

png(paste0(out,'suppression_variable_maps.png'), width=12, height=6, units="in", res=300)
par(mfrow=c(2,4), xpd=FALSE, mar=rep(0.5,4))
for(var in vars){
    p <- mapping.sprs(spdf, var, titles[which(vars==var)])
    print(p,split=c(pos[,which(vars==var)][2], pos[,which(vars==var)][1], 4, 2), newpage=FALSE) 
    print(var)
}
dev.off()

[1] "SprsCosts"
[1] "SprsAcres"
[1] "SprsCPA"
[1] "SprsFires"
[1] "PctSprs"
[1] "SprsAcre"
[1] "SprsDays"
[1] "OutDays"


pdf 
  2